In [129]:
#step one , understanding our dataset 
#Blood PRessure dataset

In [171]:
import os 
os.listdir('/kaggle/input/BloodPressureDataset')

['part_4.mat',
 'part_9.mat',
 'part_10.mat',
 'Samples',
 'part_11.mat',
 'part_3.mat',
 'part_1.mat',
 'part_8.mat',
 'part_5.mat',
 'part_6.mat',
 'part_7.mat',
 'part_2.mat',
 'part_12.mat']

In [172]:
import scipy.io
from scipy import signal 
from scipy.fftpack import fft, dct


since our data is stored in matlap files(.map)  which has the (ppg signals,the ecg signals, abp signals), we are gonna use scipy.io to read and process the signals

In [173]:
#testing on one segment of the dataset (which is one of the 12  potions /parts we have)
#loading the segment
datatest1 = scipy.io.loadmat('/kaggle/input/BloodPressureDataset/part_1.mat')

In [174]:
#lets see the ppg singal of the part1
print(datatest1.keys())

dict_keys(['__header__', '__version__', '__globals__', 'p'])


In [175]:
#lets see the size of hte dataset1
print(datatest1["p"].shape)


(1, 1000)


In [176]:
#p is the actual data  sotred in the .mat file
print(f"type of the data {type(datatest1['p'])}")

type of the data <class 'numpy.ndarray'>


In [177]:
#for the dataset1['p'] which contains the data is would be of shape 
print(datatest1['p'][0,1].shape)
#a sample one of the 1000 recordings

(3, 61000)


In [178]:
#3 rows 61000  columns 
#row 0 is ppg signals 
#row1 is blood pressure 
#rows 2 is the ecg signals 
#the 61000 refers to the number of samples per signal
print(datatest1['p'][0,1])

[[ 2.40957967  2.32942326  2.23851417 ...  1.24046921  1.23069404
   1.21994135]
 [73.75459745 73.1684682  72.53349484 ... 99.15353167 96.22288542
  93.19455095]
 [ 0.18475073  0.17986315  0.16959922 ...  0.29472141  0.31964809
   0.33968719]]


Splitting the signals into small windows for a window of size 125 (needed for time series forecasting)

In [179]:
"""data structure explanation we have datatest1 is a 1D array of length 1000 each element in this array is a matrix of dimension (3,61000) 61000 refer to the timepoitns where we stopped and mesured the values of ppg and acc and abp"""

'data structure explanation we have datatest1 is a 1D array of length 1000 each element in this array is a matrix of dimension (3,61000) 61000 refer to the timepoitns where we stopped and mesured the values of ppg and acc and abp'

In [180]:
ppg=[]
abp=[]


In [202]:
ppg_test=[]
abp_test=[]

In [203]:
#lets just see a simple exampke of how we would loop through the data we will use this example to test the model
print(datatest1['p'][0,1].shape)
for i in range(30,50):#(1,1000) size of the data of the first part  #datatest1['p'].shape[1]
            #each of the 1000 is of shape (3,61000)
            #the window size choosen is 200 (time series)
            temp_p=datatest1["p"][0,i]
            #print(temp_p.size) would be 3*61000 
            #print(temp_p.shape) #would be (3*whatever the nb of timestamps is )
            #now we segment the ppg signals 
            for j in range((int)(temp_p.shape[1]/window_size)):
                temp_ppg=temp_p[0,j*window_size:(j+1)*window_size]
                ppg_test.append(temp_ppg)
                temp_abp=temp_p[1,j* window_size]
                abp_test.append(temp_abp)
        
print("done")

    

(3, 61000)
done


In [181]:
window_size=200
ppg=[]
abp=[]

In [ ]:
#Due to limited resources on kaggle instead of using all the data we are suing small amount , but the initial amount of data is about 138888

In [182]:
#in the example above we are just using one of the 12 segments we have in the dataset 
# we can use this function to collect ppg and arp from more 
def combine_dataset(limit_parts=1): #we can leave the last 2 for validation
    for i in range(limit_parts):
        datatest1=scipy.io.loadmat(f'../input/BloodPressureDataset/part_{i+1}.mat')
        for i in range(10):#(1,1000) size of the data of the first part
            #each of the 1000 is of shape (3,61000)
            #the window size choosen is 200 (time series)
            temp_p=datatest1["p"][0,i]
            #print(temp_p.size) would be 3*61000 
            #print(temp_p.shape) #would be (3*whatever the nb of timestamps is )
            #now we segment the ppg signals 
            for j in range((int)(temp_p.shape[1]/window_size)):
                temp_ppg=temp_p[0,j*window_size:(j+1)*window_size]
                ppg.append(temp_ppg)
                temp_abp=temp_p[1,j*window_size]
                abp.append(temp_abp)
    return abp,ppg


In [185]:
abp,ppg=combine_dataset()

In [186]:
print(len(abp))
print(len(ppg))

4100
4100


In [187]:
print(len(ppg[1]))

200


In [188]:
print(abp[0])

67.0629551659967


In [189]:
#now our data is sequences of len 200 for ppg and a corresponding values of blood pressure

In [190]:
import numpy as np

### now we choose the model to create , able to 

In [191]:
#we prepare the data to pass to the lstm model
ppg_proc=ppg = np.array(ppg)
ppg_proc=ppg.reshape((ppg.shape[0], ppg.shape[1], 1))
abp=np.array(abp).reshape(-1,1)

In [205]:
ppg_test=np.array(ppg_test)
ppg_test=ppg_test.reshape((ppg_test.shape[0], ppg_test.shape[1], 1))

In [192]:
print(ppg.shape)
print(abp.shape)
print(ppg_proc.shape)

(4100, 200)
(4100, 1)
(4100, 200, 1)


In [193]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [194]:
model = Sequential([
    LSTM(50, input_shape=(200, 1)),  
    Dense(1) 
])

In [195]:
model.compile(optimizer='adam', loss='mse')

In [199]:
model.fit(ppg_proc, abp, epochs=20, batch_size=32)

Epoch 1/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - loss: 757.9166
Epoch 2/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - loss: 668.7704
Epoch 3/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - loss: 595.8824
Epoch 4/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 76ms/step - loss: 534.9821
Epoch 5/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - loss: 516.0951
Epoch 6/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - loss: 492.7922
Epoch 7/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - loss: 467.9564
Epoch 8/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - loss: 456.5410
Epoch 9/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - loss: 439.1831
Epoch 10/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - loss: 453.1559
Epoch 11/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - loss: 452.4850
Epoch 12/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 77ms/step - loss: 455.4382
Epoch 13/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step - loss: 452.9421
Epoch 14/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - loss: 445.0907
Epoch 15/

In [206]:
y_pred=model.predict(ppg_test)


85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step


In [207]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(abp_test, y_pred))
mae = mean_absolute_error(abp_test, y_pred)

In [210]:

print(f"rmese{rmse}")

rmese 20.317271764061965
